In [ ]:
# Import modules we will use
import pandas as pd
import numpy as np

# Read our data
sf_permits = pd.read_csv("Building_Permits.csv")

# Set seed for reproducibility
np.random.seed(0)

In [5]:
# look at a few rows of the nfl_data file. I can see a handful of missing data already!
sf_permits.sample(5)

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
40553,201403039652,8,otc alterations permit,03/03/2014,3732,008,400,NaN,Clementina,St,...,NaN,NaN,1.0,constr type 1,NaN,6.0,South of Market,94103.0,"(37.780460571778164, -122.40450626524974)",1334094491645
169731,201510159735,3,additions alterations or repairs,10/15/2015,2609,028,79,NaN,Buena Vista,Tr,...,5.0,wood frame (5),5.0,wood frame (5),NaN,8.0,Castro/Upper Market,94117.0,"(37.76757916496494, -122.43793170417105)",1399356139170
19180,M409787,8,otc alterations permit,07/22/2013,4624,031,178,NaN,West Point,Rd,...,NaN,NaN,NaN,NaN,NaN,10.0,Bayview Hunters Point,94124.0,"(37.73524725436046, -122.38063828309745)",1311685491725
68047,201411191888,8,otc alterations permit,11/19/2014,0039,109,294,NaN,Francisco,St,...,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,North Beach,94133.0,"(37.805257822817126, -122.40998545760392)",1362881288870
64238,M527228,8,otc alterations permit,10/14/2014,1251,002,707,NaN,Cole,St,...,NaN,NaN,NaN,NaN,NaN,5.0,Haight Ashbury,94117.0,"(37.76836885973765, -122.45074431487859)",135886493776


## See how many missing data points we have

In [10]:
# Get the number of missing data points per column
missing_values_count = sf_permits.isnull().sum()

# Look at the number of missing points in the data
missing_values_count[0:]

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

In [13]:
# How many total missing values do we have?
total_cells = np.product(sf_permits.shape)
total_missing = missing_values_count.sum()

# Percent of data that is missing
(total_missing/total_cells) * 100

26.26002315058403

As we can see, more than a quarter of our data corresponds to non-data values. Now we will analyze the data in order to know the reason for these null values.

## Figure out why the data is missing

To deal with missing values we need to resort to intuition to know why those values are missing. A fundamental question we must ask ourselves is:
#### Is this value missing because it was not recorded or because it does not exist?
If a value is missing because it does not exist (such as the age of the children of a family that has no children), there is no point in guessing what value it might be.
If a value is missing because it was not recorded, we can guess what it might have been based on the other values in that column and row. (This is known as "IMPUTATION" and we will see it below)

In [14]:
# Look at the number of missing points in the data
missing_values_count[0:]

Permit Number                                  0
Permit Type                                    0
Permit Type Definition                         0
Permit Creation Date                           0
Block                                          0
Lot                                            0
Street Number                                  0
Street Number Suffix                      196684
Street Name                                    0
Street Suffix                               2768
Unit                                      169421
Unit Suffix                               196939
Description                                  290
Current Status                                 0
Current Status Date                            0
Filed Date                                     0
Issued Date                                14940
Completed Date                            101709
First Construction Document Date           14946
Structural Notification                   191978
Number of Existing S

As we can see, we have many missing values in variables such as:

First Construction Document Date, Structural Notification, Number of Existing Stories, Number of Proposed Stories, Voluntary Soft-Story Retrofit, Fire Only Permit and Permit Expiration Date

These variables refer to situations over time. For example, the variable "Structural Notification" has 191978 which may refer to the fact that said notification has not yet arrived. Therefore, it is data that does not exist and is not related to poor data collection.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Drop missing values

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values.

In [15]:
# Remove all the rows that contain a missing value
sf_permits.dropna()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,...,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID


This is because every row in our dataset had at least one missing value. We might have better luck removing all the columns that have at least one missing value instead.

In [18]:
# Remove all columns with at least one missing value
columns_with_na_dropped = sf_permits.dropna(axis=1)
columns_with_na_dropped.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Name,Current Status,Current Status Date,Filed Date,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,Ellis,expired,12/21/2017,05/06/2015,1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,Geary,issued,08/03/2017,04/19/2016,1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,Pacific,withdrawn,09/26/2017,05/27/2016,1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,Pacific,complete,07/24/2017,11/07/2016,1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,Market,issued,12/01/2017,11/28/2016,144548169992


In [21]:
# Just how much data did we lose?
print("Columns in original dataset: %d \n" % sf_permits.shape[1])
print("Columns in original dataset: %d \n" % columns_with_na_dropped.shape[1])

Columns in original dataset: 43 

Columns in original dataset: 12 



## Filling in missing values automatically


In [25]:
# Get a small subset of the NFL dataset
subset_sf_permits = sf_permits.loc[:, 'Street Number Suffix':'Location'].head()
subset_sf_permits

,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location
0,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)"
1,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)"
2,NaN,Pacific,Av,NaN,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,NaN,...,NaN,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)"
3,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)"
4,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,NaN,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)"


In [26]:
# Replace all NA's with 0
subset_sf_permits.fillna(0)

C:\Users\usuario\AppData\Local\Temp\ipykernel_10356\1037732806.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subset_sf_permits.fillna(0)


,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location
0,0,Ellis,St,0.0,0,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,0,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)"
1,0,Geary,St,0.0,0,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,0,3.0,constr type 3,0.0,0,0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)"
2,0,Pacific,Av,0.0,0,installation of separating wall,withdrawn,09/26/2017,05/27/2016,0,...,0,1.0,constr type 1,1.0,constr type 1,0,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)"
3,0,Pacific,Av,0.0,0,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,0,5.0,wood frame (5),5.0,wood frame (5),0,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)"
4,0,Market,St,0.0,0,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,0,3.0,constr type 3,0.0,0,0,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)"


I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [27]:
# Replace all NA's the value that comes directly after it in the same column, 
# Then replace all the reamining na's with 0
subset_sf_permits.fillna(method="bfill",axis=0).fillna(0)

C:\Users\usuario\AppData\Local\Temp\ipykernel_10356\677563574.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset_sf_permits.fillna(method="bfill",axis=0).fillna(0)
C:\Users\usuario\AppData\Local\Temp\ipykernel_10356\677563574.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  subset_sf_permits.fillna(method="bfill",axis=0).fillna(0)


,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,...,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location
0,0.0,Ellis,St,0.0,0.0,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,...,0.0,3.0,constr type 3,1.0,constr type 1,0.0,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)"
1,0.0,Geary,St,0.0,0.0,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,...,0.0,3.0,constr type 3,1.0,constr type 1,0.0,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)"
2,0.0,Pacific,Av,0.0,0.0,installation of separating wall,withdrawn,09/26/2017,05/27/2016,07/18/2017,...,0.0,1.0,constr type 1,1.0,constr type 1,0.0,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)"
3,0.0,Pacific,Av,0.0,0.0,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,...,0.0,5.0,wood frame (5),5.0,wood frame (5),0.0,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)"
4,0.0,Market,St,0.0,0.0,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,...,0.0,3.0,constr type 3,0.0,0,0.0,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)"
